In [215]:
import bleach
import re

In [216]:
# Read HTML into a string
with open('list.html') as f:
    content = f.read()

In [217]:
# Use bleach to remove HTML tags
content = bleach.clean(content, tags=[], strip=True)

# Convert content to a list of strings
content = content.splitlines()

### Slice out the Data We want

Direct examination of the **list.html** indicates that the data we want is contained in a block of lines where:

- the first line in the block is the last line in the document which contains the word **Agave**
- the last line in the block is the last line in the document which contains **M=x**

In [218]:
for i, s in enumerate(content):
    if 'Agave' in s:
        first_line = i
    if 'M=x' in s:
        last_line = i

# Select the slice we want
content = content[first_line:last_line+1]

content[0:30]

['Agave (Agave sisalana)[edit]',
 'oriental scale',
 '(Aonidiella  orientalis  (Newstead))',
 'Homoptera: Diaspididae',
 'Y=x',
 'M=x',
 '',
 'grey pineapple mealybug',
 '(Dysmicoccus  neobrevipes  Beardsley)',
 'Homoptera: Pseudococcidae',
 'M=x',
 '',
 'Avocado (Persea americana)[edit]',
 'weevil',
 '(Trigonops  sp. )',
 'Coleoptera: Curculionidae',
 'B=x',
 'Y=x',
 'T=x',
 'P=x',
 'M=x',
 '',
 'phytorus leaf beetle',
 '(Phytorus  lineolatus  Weise)',
 'Coleoptera: Chrysomelidae',
 'M=x',
 '',
 'tea shot-hole borer',
 '(Xyleborus  fornicatus  Eichhoff)',
 'Coleoptera: Scolytidae']

### Generate a flat file and write it to disk.

In [219]:
i = 0
c = content
maxi = len(c)-1

# Initialize flatfile string with headers.
flatfile = 'crop|pest_common_name|pest_scientific_name|pest_order|pest_family|pest_distribution\n'

while i <= maxi:
    if '[edit]' in c[i]:
        crop = c[i]
        crop = crop.replace('[edit]', '')
        i = i + 1
    else:
        s = ''
        while (i <= maxi) and (c[i] != ''):
            s = s + c[i]
            if not ('=' in c[i]):
                 s = s + '|'
            i = i + 1
            
        # A bit of clean up
        s = s.replace('  ', ' ') # Get rid of double spaces
        s = s.replace(': ', '|') # Split pest order and family
        
        s = re.sub('&lt;.*?&gt;','XX',s) # Remove lingering anchor tags
        s = s.replace('XX', '|')
        
        if s != '':
            flatfile += crop + '|' + s + '\n'
            flatfile = flatfile.replace('||', '|')
        i = i + 1
        
# Write flatfile to disk
with open("crop-pest.csv", "w") as f:
    f.write(flatfile)

### Read lines from disk and check that each tine contains 6 fiedls (5 '|'s)

In [220]:
# Read flatfile from disk
with open("crop-pest.csv", "r") as f:
    ff = f.readlines()

In [221]:
for i, s in enumerate(ff):
    n = s.count('|')
    if n != 5:
        print(i, ' ', n, ' ', s)

60   9   Beans (Phaseolus)|broad mite (Polyphagotarsonemus latus (Banks)) Acari|Tarsonemidae M=xspider mite (Tetranychus tumidus) Acari|Tetranychidae M=xleaf beetle (Pagria signata) (Motschulsky) Coleoptera|Chrysomelidae M=G chinese rose beetle (Adoretus sinicus Burmeister) Coleoptera|Scarabaeidae B=x M=x bean fly|(Ophiomyia phaseoli (Tryon))|Diptera|Agromyzidae|B=xY=xT=xM=x

99   6   Beans (Phaseolus)|==Bermuda grass (Cynodon)|rhodesgrass mealybug|(Antonina graminis (Maskell))|Homoptera|Pseudococcidae|B=xT=xM=x

102   1   Betel nut, Areca|

127   2   Betel nut, Areca|Back to Crop List|

366   4   Cycads, Cycas|caterpillar|(Chilades pandava )|Lepidoptera:Laecinidae|M=S



### Great!!! We only have 5 problem lines so let's fix them here.

In [222]:
new = [] # New records to replace problem records
removal_list = [] # Indices of records to be removed
removal_list.append(0) # Add header row to removal list; will add this after sorting

# Replace ff[60] with:
removal_list.append(60)
new.append('Beans (Phaseolus)|broad mite (Polyphagotarsonemus latus (Banks)) Acari|Tarsonemidae M=x')
new.append('Beans (Phaseolus)|spider mite (Tetranychus tumidus) Acari|Tetranychidae M=x')
new.append('Beans (Phaseolus)|leaf beetle (Pagria signata) (Motschulsky) Coleoptera|Chrysomelidae M=G')
new.append('Beans (Phaseolus)|chinese rose beetle (Adoretus sinicus Burmeister) Coleoptera|Scarabaeidae B=x M=x') 
new.append('Beans (Phaseolus)|bean fly|(Ophiomyia phaseoli (Tryon))|Diptera|Agromyzidae|B=xY=xT=xM=x')

# Replace ff[99] with:
removal_list.append(99)
new.append('Bermuda grass (Cynodon)|\
rhodesgrass mealybug|Antonina graminis (Maskell)|Homoptera|Pseudococcidae|B=xT=xM=x')

# Replace ff[100] with:
removal_list.append(100)
new.append('Bermuda grass (Cynodon)|\
grass leaf-folder|(Marasmia venilialis (Walker))|Lepidoptera|Pyralidae|M=G')

# Replace ff[101] with:
removal_list.append(101)         
new.append('Bermuda grass (Cynodon)|lawn armyworm|(Spodoptera mauritia Guenée)|Lepidoptera|Noctuidae|M=x')

# Delete 102, 127
removal_list.append(102)
removal_list.append(127)

# Replace ff[366] with:
removal_list.append(366)
new.append('Cycads, Cycas|caterpillar|(Chilades pandava )|Lepidoptera|Lycaenidae|M=S')
           
for i, s in enumerate(ff):
    if not (i in removal_list):
           new.append(s)

new = sorted(new)
new.insert(0, ff[0])

with open('crop-pest.csv', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(new))

### Let's load the data into a Pandas data frame

In [223]:
import pandas as pd
df = pd.read_csv('crop-pest.csv', sep='|', header=0)

# Remove parentheses around pest_scientific_name
for i, s in enumerate(df.pest_scientific_name):
    s = re.sub('^\(', '', s)
    s = re.sub('\)$', '', s)
    df.pest_scientific_name[i] = s
    
df.pest_scientific_name
df.to_csv('crop-pest.csv', index=False)
df

,crop,pest_common_name,pest_scientific_name,pest_order,pest_family,pest_distribution
0,Agave (Agave sisalana),grey pineapple mealybug,Dysmicoccus neobrevipes Beardsley,Homoptera,Pseudococcidae,M=x
1,Agave (Agave sisalana),oriental scale,Aonidiella orientalis (Newstead),Homoptera,Diaspididae,Y=xM=x
2,Avocado (Persea americana),coconut scale,Aspidiotus destructor Signoret,Homoptera,Diaspididae,B=xY=xT=xP=xM=x
3,Avocado (Persea americana),egyptian fluted scale,Icerya aegyptiaca (Douglas),Homoptera,Margarodidae,B=xY=xT=xP=xM=x
4,Avocado (Persea americana),florida wax scale,Ceroplastes floridensis Comstock,Homoptera,Coccidae,B=xP=xM=x
5,Avocado (Persea americana),latania scale,Hemiberlesia lataniae (Signoret),Homoptera,Diaspididae,B=xY=xT=xP=xK=xM=x
6,Avocado (Persea americana),long brown scale,Coccus longulus (Douglas),Homoptera,Coccidae,B=xM=x
7,Avocado (Persea americana),long-tailed mealybug,Pseudococcus longispinus (Targioni-Tozzetti),Homoptera,Pseudococcidae,T=?
8,Avocado (Persea americana),melon fly,Dacus cucurbitae Coquillett,Diptera,Tephritidae,M=x
9,Avocado (Persea americana),palm scale,Hemiberlesia palmae (Cockerell),Homoptera,Diaspididae,B=xY=xT=YP=xK=xM=x


### FINISHED
To read the csv file into a Pandas data frame use: 

    df = pd.read_csv('crop-pest.csv', header=0)